#### LLM Introduction

In [66]:
import json
from tqdm import tqdm
from typing import Optional
from tiktoken import encoding_for_model

In [4]:
with open('../documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

##### Index Data

In [8]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

In [10]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [11]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}  ### Make the field a keyword to query on
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [15]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc) # create a record in the index for each document

  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [00:04<00:00, 200.00it/s]


#### Search the index

In [50]:
def elastic_search(query: str, course_filter: Optional[str] = None, n_results: int = 1):
    if course_filter:
        search_query = {
            "size": n_results, #how many index records to return
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^4", "text"], #boost is determined by exponent "^"
                            "type": "best_fields"
                        }
                    },
                    "filter": {
                        "term": {
                            "course": "machine-learning-zoomcamp"
                        }
                    }
                }
            }
        }

    else:
        search_query = {
            "size": n_results, #how many index records to return
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^4", "text"], #boost is determined by exponent "^"
                            "type": "best_fields"
                        }
                    },
                }
            }
        }
        
    response = es_client.search(index=index_name, body=search_query) #search the index for relevant documents based on search settings

    result_docs = []
    for hit in response['hits']['hits']:
        source_dict = hit['_source']
        source_dict['score'] = hit['_score']
        result_docs.append(hit['_source'])

    return result_docs



In [54]:
question_query = 'How to execute a command on a Kubernetes pod?'
elastic_search(query=question_query)

[{'text': 'Problem description:\nI started a web-server in terminal (command window, powershell, etc.). How can I run another python script, which makes a request to this server?\nSolution description:\nJust open another terminal (command window, powershell, etc.) and run a python script.\nAlena Kniazeva',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How to run a script while a web-server is working?',
  'course': 'machine-learning-zoomcamp',
  'score': 38.491016}]

In [62]:
# Filtering
question_query = "How do copy a file to a Docker container?"
docker_question_search_results =  elastic_search(query=question_query, course_filter="machine-learning-zoomcamp", n_results= 3)

#### Building a Prompt

In [63]:
def build_prompt(query, search_results):
    
    context = ""
    for doc in search_results: 
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    
    prompt = f"""
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the question.

    QUESTION: {query} 

    CONTEXT: {context}

    """.strip()
    
    return prompt

In [64]:
question_query = "How do copy a file to a Docker container?"
docker_prompt = build_prompt(query=question_query, search_results=docker_question_search_results)

In [65]:
len(docker_prompt)

1629

#### Prompting using tokens

In [67]:
encoding = encoding_for_model('gpt-4o')

In [71]:
token_lst = encoding.encode(docker_prompt)
len(token_lst)

356

In [72]:
encoding.decode_single_token_bytes(63842)

b"You're"

### Send the prompt to openAI

In [75]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
api_key = os.getenv('API_KEY')

In [76]:
client = OpenAI()

In [77]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": docker_prompt}]
)

print(response.choices[0].message.content)

You can copy a file to a Docker container using the `docker cp` command. The basic syntax is as follows:

```bash
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```


#### Calculate the cost

In [78]:
input_tokens = len(encoding.encode(docker_prompt))
output_tokens = len(encoding.encode(response.choices[0].message.content))

input_price_per_1k_tokens = 0.005
output_price_per_1k_tokens = 0.015

price = input_tokens / 1_000 * input_price_per_1k_tokens + output_tokens / 1_000 * output_price_per_1k_tokens

print(price)

0.00244
